In [3]:
from time import time
import random


def gcd(e, totient):
    """
    Verify that e and totient are coprime
    If totient equals 0 then we return e 
    Else we call the gcd function again with the totient and e % totient --> To compute the remainder 
    And we do that until we have totient equals 0
    """
    if totient == 0:
        return e
    else:
        return gcd(totient, e % totient)


def bezout_identity(e, totient):
    """
    The bezout identity (Extended Euclidean algorithm)
    Generates two coefficient such that e * a - totient * b = 1

    This function help us to find the inverse of e mod totient directly
    We return the first coefficient of the equation
    """
    L1 = [e, 1, 0]
    L2 = [totient, 0, 1]

    while L2[0] > 0:
        L = L2[:]
        q = L1[0] // L2[0]

        for i in range(0, 3):
            L2[i] = L1[i] - q * L2[i]
        L1 = L

    return L1[1]

def generates_e():
    """
    Generates the number e such that 100<e<65536 --> 100 for minimal security purpose, 65536 for performance purpose
    Balance between security and performance
    To verify if e is coprime with totient, we paste it in a gcd function
    A number are coprime if his gcd equals 1 
    """
    while True:
        e = random.randrange(100, 65536)

        if gcd(e, totient) == 1:
            return e

def generates_d():
    """
    Generates d, d is use to generate the private key.
    d = inverse(e) mod totient --> bezout identity find directly inverse(e) mod totient
    If coef a is < 0 the we add the totient to have a positive number
    """
    coef_a = bezout_identity(e, totient)
    # make sure d is positive
    if coef_a < 0:
        d = coef_a + totient
    else:
        d = coef_a

    return d

def modexp(base, power, mod):
    """
    Compute base^power % mod
    Because power is very large, we need to use square & multiple algorithm
    """
    x = 1
    while power > 0:
        base, power, x = (
            base * base % mod,
            power // 2,
            base * x % mod if power % 2 else x
        )

    return x

def write_keypair():
    """
    Write the public key and the private key in files
    The public key = n + e
    The private key = n + d
    """
    f_public = open('./public_keys.txt', 'w')
    f_public.write(str(n) + str(e))
    f_public.close()
    f_private = open('./private_keys.txt', 'w')
    f_private.write(str(n) + str(d))
    f_private.close()

def write_ciphertext(ciphertext):
    """Create a ciphertext.txt file and paste it the ciphertext, we have a linebreak for each letter"""
    f_ciphertext = open('./ciphertext.txt', 'w')
    for element in ciphertext:
        f_ciphertext.write(str(element) + "\n")

    f_ciphertext.close()

def write_plaintext(plaintext):
    """
    We create a plaintext.txt file
    We recover the ascii characters of plaintext and we convert them to string
    Then we write the results in the file
    """
    plaintext_recover = [chr(c) for c in plaintext]
    f_plaintext = open('./plaintext.txt', 'w')
    f_plaintext.write(str("".join(plaintext_recover)))
    f_plaintext.close()
    
def write_plaintext_attack(plaintext):
    """
    Same method than write_plaintext function
    The only difference here is that we create a new variable and add all the element in this string variable
    Then we paste the results in the plaintext_attack.txt file
    """
    ascii_character = plaintext.splitlines()
    plaintext_recover = ""
        
    for element in ascii_character:
        plaintext_recover += chr(int(element))
    
    f_plaintext = open('./plaintext_attack.txt', 'w')
    f_plaintext.write(plaintext_recover)
    f_plaintext.close()

def rsa_enc(original_plaintext):
    """
    Encryption of the original plaintext
    c = m^e mod n
    """
    ciphertext = [modexp(ord(c), e, n) for c in original_plaintext]

    return ciphertext

def rsa_dec(ciphertext):
    """
    Decryption of the original plaintext
    m = c^d mod n
    """
    plaintext = [modexp(int(c), d, n) for c in ciphertext]
    
    return plaintext

def attack(ciphertext):
    """
    Generate a random number r such that r != 1 mod n
    Generate a fake ciphertext with the true ciphertext
    After that, we call the rsa_dec function to decrypt the fake ciphertext
    This return a fake plaintext
    Finally, to recover the plaintext, we put the (fake plaintext / r) mod n
    """
    r = random.randint(2, 5)
    ciphertext_lines = ciphertext.splitlines()
    ciphertext_prime = [int(c) * modexp(r, e , n) for c in ciphertext_lines]


    plaintext_prime = (rsa_dec(ciphertext_prime))

    plaintext_recover = ""
        
    for i in range(len(plaintext_prime)):
        plaintext_recover += str((plaintext_prime[i] / r) % n)
        plaintext_recover += "\n"
            
    return plaintext_recover
   

if __name__ == '__main__':
    print(
"""
###########################################
# RSA Implementation  &                   #
# Chosen Ciphertext Attack                #
# By Edouard Bettignies Masi-M1           #
###########################################
"""
    )
    p = 147513732454791286855894116408250922648769471923036927880790753919871952672308331279550605377807291710348597515184859660202801025967131853934063539004405289481737363292768134297535316056328161832744772210538303782101006513784097226420866643633546813327763529407492394840771466569824419814700456992846334731281

    q = 150735041362626454417511646300711556141544472374851217978606636468341071806150452400352225218785400357093050355924724149364822513948644422533623676488140164031920481113890696128286021327454231109796167684696885749396571704094576432688322687991081472100194231663588044246927950986278951923846041547982212150073

    n = p * q
    totient = (p - 1) * (q - 1)

    e = generates_e()
    
    d = generates_d()    

    print(f"n = {n}\n")
    print(f"totient = {totient}\n")
    print(f"e = {e}\n")
    print(f"d = {d}\n")
    
    print("Key pair generation begins...")
    write_keypair()
    print("Key pair generation ends...\n")
    print("Your public key is in the file public_keys.txt")
    print("Your private key is in the file private_keys.txt\n")

    choose_mode = input("Do you want encrypt/decrypt or attack ? (e/a) ")
    start_time = time()


    f = open("./original_plaintext.txt" , "r")
    original_plaintext_file = f.read()
    print("Encryption begins...")
    write_ciphertext(rsa_enc(original_plaintext_file))
    print("Encryption ends in {:.5f} seconds \n".format(time() - start_time))
    print("The ciphertext are available in the file ciphertext.txt !\n")

    start_time_2 = time()
    
    if choose_mode == "e":
        f = open("./ciphertext.txt", "r")
        ciphertext_file = f.read()
        print("Decryption begins...")
        ciphertext_lines = ciphertext_file.splitlines()
        write_plaintext(rsa_dec(ciphertext_lines))
        print("Decryption ends in {:.5f} seconds \n".format(time() - start_time_2))
        print("The plaintext are available in the file plaintext.txt !")

        
    elif choose_mode == "a":
        f = open("./ciphertext.txt", "r")
        ciphertext_file = f.read()
        print("Attack begins...")
        write_plaintext_attack(attack(ciphertext_file))
        print("Attack end in {:.5f} seconds \n".format(time() - start_time_2))
        print("The plaintext are available in the file plaintext_attack.txt !")



###########################################
# RSA Implementation  &                   #
# Chosen Ciphertext Attack                #
# By Edouard Bettignies Masi-M1           #
###########################################

n = 22235488563128377048640341680268782796727601669579998295472069462597227101092882591890912718645362758254105343115830686504026878622237714960954535018600197167127452159027435054411871291972850272494257816808434496029073062018118729829903373281327432865226068296143908691880391130296423048712714607456846134140473097475691095776669368612208146002243006984457796591187129201259084507561370145417786300110247259930765413964497509459731474099562995451508436149183159151456822123629926943878300949313887437090000226030931909228596811656947661415168112177733046429406258234198125986620844853989924288095072948215965699533513

totient = 2223548856312837704864034168026878279672760166957999829547206946259722710109288259189091271864536275825410534311583068650402687862223771496095453